In [0]:
from huggingface_hub import notebook_login

notebook_login()

In [0]:
import json
from datasets import load_dataset


output_model_path = "/dbfs/fine_tuned_bge_v1_5_model"

dataset = load_dataset("embedding-data/Amazon-QA", split = 'train[:500]')

dataset[2]

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'query': 'What is the length and width of this scooter in inches?',
 'pos': ['below are all the products specs- We are authorized dealers and offer fast shipping- Also 1 year warranty FREE Phillips Health care he EW-36 mobility scooter is a 3 wheel high-power mobility scooter using a transaxle brushless electric motor. At full power the EW-36 can reach speed up to 18 mph which makes it easily the fastest mobility scooter on the market! Up to 45 mile range per single charge! Also included a reverse switch, armrest and large storage basket. The EW-36 comes equipped with a digital anti-theft alarm and two rear shock absorbers on the frame. The EW-36 is a great scooter for both fun and transportation. COLOR: Red, Silver, Blue, Orange, Camo POWER: Electric WATTS: 500 Watt MOTOR TYPE: Transaxle differential brushless motor AMPS: 20 AH VOLTS: 48 Volt DIMENSIONS: 61"*29"*41" BATTERIES: 48 volt lead acid maintenance free battery TIRE SIZE: 16"/2.5" CHARGER: Smart charger included SPEED: Up to 

In [0]:
from sentence_transformers import InputExample


def create_dataset_for_multiple_loss(train_dataset):
    train_examples = []
    for elem in train_dataset:
        query = f"Represent this sentence for searching relevant passages: {elem['query']}"
        texts = elem["pos"]

    for text in texts:
        train_examples.append(InputExample(texts=[query,text]))
    return train_examples

train_examples = create_dataset_for_multiple_loss(dataset)


2023-12-02 05:38:01.451378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 05:38:01.451484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 05:38:01.457287: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 05:38:02.680530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

from torch.utils.data import DataLoader
from sentence_transformers import losses
from accelerate import notebook_launcher

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=3)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [0]:
model.save(output_model_path)

In [0]:
import mlflow
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
 
class SentenceTransformerEmbeddingModel(mlflow.pyfunc.PythonModel):
  def load_context(self, context):
    device = 0 if torch.cuda.is_available() else -1
    self.model = SentenceTransformer(context.artifacts["sentence-transformer-model"], device=device)
    
  def predict(self, context, model_input): 
    texts = model_input.iloc[:, 0].to_list()  # get the first column
    sentence_embeddings = self.model.encode(texts)
    return pd.Series(sentence_embeddings.tolist())

In [0]:
from mlflow.utils.environment import _mlflow_conda_env
import accelerate
import sentence_transformers
import cloudpickle
EMBEDDING_CONDA_ENV = _mlflow_conda_env(
    additional_pip_deps=[
        f"accelerate=={accelerate.__version__}",
        f"cloudpickle=={cloudpickle.__version__}",
        f"sentence-transformers=={sentence_transformers.__version__}",
    ]
)

/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [0]:
import mlflow

with mlflow.start_run() as run:
  my_model = SentenceTransformerEmbeddingModel()
  model_info = mlflow.pyfunc.log_model(artifact_path="model", python_model=my_model, input_example=["London is known for its finacial district"], artifacts={"sentence-transformer-model": output_model_path}, conda_env=EMBEDDING_CONDA_ENV)

2023/12/02 05:39:04 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: AttributeError("'list' object has no attribute 'iloc'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.


2023/12/02 05:39:04 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading artifacts:   0%|          | 0/18 [00:00<?, ?it/s]

2023/12/02 05:39:09 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading /local_disk0/repl_tmp_data/ReplId-6c9d2-23938-df28d-4/tmpbo4yqxsj/model/artifacts/fine_tuned_bge_v1_…

In [0]:
import mlflow
import pandas as pd

# Load model as a PyFuncModel.
run_id = run.info.run_id
logged_model = f"runs:/{run_id}/model"

loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
test_df = pd.DataFrame(['London has 9,787,426 inhabitants at the 2011 census',
              'London is known for its finacial district'], columns=["text"])

loaded_model.predict(test_df)

2023/12/02 05:44:52 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
File <command-2525483273307466>, line 8
      5 run_id = run.info.run_id
      6 logged_model = f"runs:/{run_id}/model"
----> 8 loaded_model = mlflow.pyfunc.load_model(logged_model)
     10 # Predict on a Pandas DataFrame.
     11 test_df = pd.DataFrame(['London has 9,787,426 inhabitants at the 2011 census',
     12               'London is known for its finacial district'], columns=["text"])

File /databricks/python/lib/python3.10/site-packages/mlflow/pyfunc/__init__.py:679, in load_model(model_uri, suppress_warnings, dst_path, model_config)
    677         model_impl = importlib.import_module(conf[MAIN])._load_pyfunc(data_path, model_config)
    678     else:
--> 679         model_impl = importlib.import_module(conf[MAIN])._load_pyfunc(data_path)
    680 except ModuleNotFoundError as e:
    681     if conf[MAIN] == _DATABRICKS_FS_LOAD

In [0]:
test_df = pd.DataFrame(['London has 9,787,426 inhabitants at the 2011 census',
                        'London is known for its finacial district'], columns=["text"])

# Add the instruction to each entry in the "text" column
test_df['text'] = 'Represent this sentence for searching relevant passages: ' + test_df['text']
loaded_model.predict(test_df)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-2525483273307464>, line 6
      4 # Add the instruction to each entry in the "text" column
      5 test_df['text'] = 'Represent this sentence for searching relevant passages: ' + test_df['text']
----> 6 model.predict(test_df)

File /databricks/python/lib/python3.10/site-packages/torch/nn/modules/module.py:1614, in Module.__getattr__(self, name)
   1612     if name in modules:
   1613         return modules[name]
-> 1614 raise AttributeError("'{}' object has no attribute '{}'".format(
   1615     type(self).__name__, name))

AttributeError: 'SentenceTransformer' object has no attribute 'predict'

In [0]:
model.save_to_hub(
    repo_name ="01GangaPutraBheeshma/01databricksmlftbgelargeenUT",
    organization="Sentence Transformers",
    commit_message = "Add a new SentenceTransformer Model from Databricks",
    exist_ok = False, 
    replace_model_card= False, 
    train_datasets = "embedding-data/Amazon-QA")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-2525483273307456>, line 1
----> 1 model.save_to_hub(
      2     repo_name ="01GangaPutraBheeshma/01databricksmlftbgelargeenUT",
      3     organization="Sentence Transformers",
      4     commit_message = "Add a new SentenceTransformer Model from Databricks",
      5     exist_ok = False, 
      6     replace_model_card= False, 
      7     train_datasets = "embedding-data/Amazon-QA")

File /databricks/python/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:464, in SentenceTransformer.save_to_hub(self, repo_name, organization, private, commit_message, local_model_path, exist_ok, replace_model_card, train_datasets)
    462         repo_name = splits[1]
    463     else:
--> 464         raise ValueError("You passed and invalid repository name: {}.".format(repo_name))
    466 endpoint = "https://hu

In [0]:
save_to_hub(repo_name: str, organization: Optional[str] = None, private: Optional[bool] = None, commit_message: str = 'Add new SentenceTransformer model.', local_model_path: Optional[str] = None, exist_ok: bool = False, replace_model_card: bool = False, train_datasets: Optional[List[str]] = None)